<a href="https://colab.research.google.com/github/Sejong-Kaggle-Challengers/jeongmin/blob/main/%ED%87%B4%EA%B7%BC%EC%8B%9C%EA%B0%84_%EB%B2%84%EC%8A%A4%EC%8A%B9%EC%B0%A8%EC%9D%B8%EC%9B%90%EC%98%88%EC%B8%A1_1%EB%93%B1_%EC%BD%94%EB%93%9C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#날씨데이터 크롤링
- 2019년 9월 1일부터 2019년 10월 16일 
- 오전 10시 관측 데이터
- http://www.weather.go.kr/

In [1]:
!pip install selenium

     |████████████████████████████████| 911kB 5.3MB/s 


In [2]:
!apt-get update # apt install을 정확히 실행하기 위해 설치된 패키지 업데이트
!apt install chromium-chromedriver #패키지 설치
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

#cp : 유닉스 셸에서 한 파일을 어떤 장소에서 다른 장소로 또는 다른 파일 시스템으로 옮길때 사용하는 명령어
#/usr/lib/chromium-browser/chromedriver 파일을 /usr/bin로 복사한다.

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [44.4 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
'''
다른 폴더(경로)에 있는 python 파일 import
경로를 환경변수로 지정 --> 경로를 환경변수로 설정하게 해주면 그 경로 안의 프로그램을 실행시킬 때 마다
경로를 매번 입력하지 않아도 된다.
'''
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
#chrome의 option 설정
'''
Chrome Headless 
: Chrome 브라우저를 GUI 없이 사용할 수 있는 모드
리눅스 서버와 같이 GUI를 제공하지 않는 환경에서 '--no-sandbox' 옵션 추가
disable-dev-shm-usage도 마찬가지로 GUI를 사용할 수 없어서 추가하는 것 같다.
참고 : https://go-madhat.github.io/chrome-headless/
'''
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #chrome 창이 나오지 않게 해준다.
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#1등 코드에서의 browser == driver
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: use options instead of chrome_options


In [4]:
import pandas as pd
import time

def crawl_weather() :
  weather_data_10 = pd.DataFrame(columns = ['현재일기_10','현재기온_10','체감온도_10','일강수_10'])
  url = 'http://www.weather.go.kr/weather/observation/currentweather.jsp?auto_man=m&type=t99&reg=184&tm=2019.10.25.16%3A00&x=19&y=7'
  driver.get(url)

  for i in range(0, 46):
    i+=1

    #id를 찾고 그 공간을 비운 후 새로운 값을 넣어준다 (날짜값,10시)
    elem = driver.find_element_by_id('observation_text')
    elem.clear()
    elem.send_keys("2019.9.{}.10:00".format(i))

    btn = driver.find_elements_by_class_name('btn')
    btn[2].click()

    time.sleep(1)
    weathers = driver.find_elements_by_css_selector('td')
    weather_data_10 = weather_data_10.append(pd.DataFrame([[weathers[40].text,weathers[44].text, weathers[46].text, weathers[47].text]],columns=['현재일기_10','현재기온_10','체감온도_10','일강수_10']))

  print('success !')
  driver.close()

  return weather_data_10

In [5]:
weather_data = crawl_weather()
weather_data.to_csv('weather.csv', index = False)

print('save.. !')

weather_data = pd.read_csv('weather.csv')

success !
save.. !


# 라이브러리 및 데이터

In [6]:
!unzip data13.zip

Archive:  data13.zip
  inflating: 행정_법정동 중심좌표.xlsx  
  inflating: test.csv                
  inflating: train.csv               
  inflating: submission_sample.csv   
  inflating: bus_bts.csv             


In [7]:
# base
import os
import shutil
import random
import numpy as np
import time
from collections import Counter
from sklearn.metrics import mean_squared_error

# data preprocessing
import pandas as pd
pd.set_option('max_columns', 130, 'max_rows', 30)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

# ignore warining
import warnings
warnings.filterwarnings('ignore')


import folium # 지도 관련 시각화
from folium.plugins import MarkerCluster #지도 관련 시각화
import geopy.distance #거리 계산해주는 패키지 사용


# save
from sklearn.externals import joblib 
import pickle

# selenium
from selenium.webdriver import Chrome

import geopy.distance

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals import joblib 
import pickle

import random
random.seed(42)

import lightgbm as lgb
from keras import metrics
from sklearn.metrics import r2_score

In [8]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
bts = pd.read_csv("bus_bts.csv")

train.shape, test.shape, bts.shape

((415423, 21), (228170, 20), (2409414, 13))

#데이터 전처리

In [9]:
#weekday(요일) 변환
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday

In [10]:
#route_station = bus_route_id(노선ID) + station_code(해당 승하차 정류소의 ID)
train['bus_route_id'] = train['bus_route_id'].astype(str)
train['station_code'] = train['station_code'].astype(str)
train['route_station'] = train['bus_route_id'] + ',' + train['station_code']

test['bus_route_id'] = test['bus_route_id'].astype(str)
test['station_code'] = test['station_code'].astype(str)
test['route_station'] = test['bus_route_id'] + ',' + test['station_code']

In [11]:
#bus_route_id_weekday = bus_route_id(노선ID) + weekday(요일)
train['bus_route_id_weekday'] = train['bus_route_id'].astype(str) + ',' + train['weekday'].astype(str) 
test['bus_route_id_weekday'] = test['bus_route_id'].astype(str) + ',' + test['weekday'].astype(str) 

In [12]:
#station_code_weekday = station_code(해당 승하차 정류소의 ID) + weekday(요일)
train['station_code_weekday'] = train['station_code'].astype(str) + ',' + train['weekday'].astype(str)
test['station_code_weekday'] = test['station_code'].astype(str) + ',' + test['weekday'].astype(str)

In [13]:
#route_station_weekday = route_station(노선+정류소 ID) + weekday(요일) 
train['route_station_weekday'] = train['route_station'].astype(str) + ',' + train['weekday'].astype(str) 
test['route_station_weekday'] = test['route_station'].astype(str) + ',' + test['weekday'].astype(str)

In [14]:
#승하차 시간대 통합변수 ---> 1시간대 승/하차인원을 합하여 t~t+2 시간대 승/하차인원 변수를 만듬
train['68a']=train['6~7_ride']+train['7~8_ride'] 
train['810a']=train['8~9_ride']+train['9~10_ride']
train['1012a']=train['10~11_ride']+train['11~12_ride']

train['68b']=train['6~7_takeoff']+train['7~8_takeoff'] 
train['810b']=train['8~9_takeoff']+train['9~10_takeoff']
train['1012b']=train['10~11_takeoff']+train['11~12_takeoff']

test['68a']=test['6~7_ride']+test['7~8_ride']
test['810a']=test['8~9_ride']+test['9~10_ride']
test['1012a']=test['10~11_ride']+test['11~12_ride']

test['68b']=test['6~7_takeoff']+test['7~8_takeoff']
test['810b']=test['8~9_takeoff']+test['9~10_takeoff']
test['1012b']=test['10~11_takeoff']+test['11~12_takeoff']

##on_time
bts.csv 데이터에서 geton_time 열에서 시간대만 추출하여 bts.csv 안에 on_time 컬럼을 만듬

In [15]:
bts['geton_time']

0          06:34:45
1          06:34:58
2          07:19:07
3          09:14:47
4          09:28:53
             ...   
2409409    07:08:31
2409410    07:16:31
2409411    08:29:05
2409412    08:40:32
2409413    08:40:35
Name: geton_time, Length: 2409414, dtype: object

In [16]:
bts['on_time']  = bts['geton_time'].apply(lambda x : x[:2])

#13은 on_time 행
bts.iloc[bts.query('on_time == "06"').index,13] = '6~7_ride'
bts.iloc[bts.query('on_time == "07"').index,13] = '7~8_ride'
bts.iloc[bts.query('on_time == "08"').index,13] = '8~9_ride'
bts.iloc[bts.query('on_time == "09"').index,13] = '9~10_ride'
bts.iloc[bts.query('on_time == "10"').index,13] = '10~11_ride'
bts.iloc[bts.query('on_time == "11"').index,13] = '11~12_ride'

##Make features by using target variable
최종적으로 예측해야할 것은 각 일자별(date), 버스 노선(bus_route_id) 상의 정류장(station_name)의 퇴근시간 승차인원(18~20_ride)이다.
- bus_route_id, station_name, weekday의 각 조합별 퇴근시간 하차인원(18~20_ride)의 여러 통계량을 구한 후 이를 train set, test set에 모두 적용한다.
- target 변수를 train, test set에 적용할 수 있는 이유는 우리가 예측해야할 id는 date, bus_rout_id, station_name으로 구성되어있기 때문이다. 즉, 각각의 노선, 정류장별로 공통적인 패턴이 존재할 수 있다.
- 이 과정에서 NA 값이 생기는 이유는 train set에 없는 bus_route_id, station_name이 존재하기 때문

In [17]:
#각 행(ID)별 '18~20_ride'의 mean과 sum을 구하고 ID에 맞춰서 행에 추가해준다.
def id_statistic(ID, col1, col2) :
    
    # mean, sum
    rs_mean = train.groupby([ID])['18~20_ride'].agg([(col1, 'mean')]).reset_index()
    rs_sum = train.groupby([ID])['18~20_ride'].agg([(col2, 'sum')]).reset_index()
    rs_mean_sum = pd.merge(rs_mean, rs_sum, on=ID)

    # merge
    tr = pd.merge(train, rs_mean_sum, how='left', on=ID)
    te = pd.merge(test, rs_mean_sum, how='left', on=ID)

    #test의 nan값 mean으로 채워주기
    # na -&gt; mean
    te[col1] = te[col1].fillna(rs_mean.mean())
    te[col1] = te[col1].fillna(rs_sum.mean())
    
    return tr, te

In [18]:
#route_station, bus_route_id, station_code, weekday에 대해서 실행
train, test = id_statistic('route_station', '1820_rs_mean', '1820_rs_sum')
train, test = id_statistic('bus_route_id', '1820_r_mean', '1820_r_sum')
train, test = id_statistic('station_code', '1820_s_mean', '1820_s_sum')
train, test = id_statistic('weekday', '1820_w_mean', '1820_w_sum')
train.shape, test.shape

((415423, 40), (228170, 39))

In [19]:
#위의 함수와 거의 동일한 것 같다. mean값만 구한다. Nan값 처리가 따로 없다.
def mean_statistics() :

    f = train.groupby(['bus_route_id_weekday'])['18~20_ride'].agg([('mean_bus_weekday_ride','mean')]).reset_index()
    tr = pd.merge(train, f, how='left', on='bus_route_id_weekday')
    te = pd.merge(test, f, how='left', on='bus_route_id_weekday').fillna(f['mean_bus_weekday_ride'].mean())
    
    f = train.groupby(['station_code_weekday'])['18~20_ride'].agg([('mean_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='station_code_weekday')
    te = pd.merge(te, f, how='left', on='station_code_weekday').fillna(f['mean_station_weekday_ride'].mean())
    
    f = train.groupby(['route_station_weekday'])['18~20_ride'].agg([('mean_route_station_weekday_ride','mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='route_station_weekday')
    te = pd.merge(te, f, how='left', on='route_station_weekday').fillna(f['mean_route_station_weekday_ride'].mean())
    
    return tr, te

In [20]:
train, test = mean_statistics()
train.shape, test.shape

((415423, 43), (228170, 42))

###혼잡도 계산

In [21]:
def congestion() :
    df = train.groupby(['bus_route_id'])['18~20_ride'].agg([('passenger', 'sum')])
    df = df.sort_values(by='passenger', ascending=False).reset_index()
    
    def f(x):
        if x > 10000:
            return 7
        elif x > 5000:
            return 6
        elif x > 2000:
            return 5
        elif x > 700:
            return 4
        elif x > 200:
            return 3
        elif x > 50:
            return 2
        else:
            return 1
    
    df['congestion']=df['passenger'].apply(f)
    df = df[['bus_route_id','congestion']]
    
    tr = pd.merge(train, df, how='left', on='bus_route_id')
    te = pd.merge(test, df, how='left', on='bus_route_id')
    
    # 결측치는 데이터 프레임 df의 'congestion'의 중간값인 '4'으로 대체
    te = te.fillna(4)
    
    return tr, te

In [22]:
train, test = congestion()

#데이터 전처리 이어서

In [23]:
#location = latitude + longitude
train['location'] = train['latitude'].astype(str) + ',' + train['longitude'].astype(str)
test['location'] = test['latitude'].astype(str) + ',' + test['longitude'].astype(str)

In [24]:
#merge key --> train과 test 두개를 합쳐서 하는 곳에서 구분 위해 만든 것 같다.
# make cue column
train['cue']=0
test['cue']=1

## 오전 시간 여러 데이터 활용 변수

오전 시간의 탑승 및 하차 데이터를 활용하여 요약 통계량을 만든다.

In [25]:
def morning() :
    
    # merge
    data = pd.concat([train, test])
    
    a = data.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()
    a.columns = ['route_station', '1012a_sum','1012a_mean']

    b = data.groupby(['route_station'])['1012b'].agg({'sum', 'mean'}).reset_index()
    b.columns = ['route_station', '1012b_sum','1012b_mean']
    b = b[['1012b_sum','1012b_mean']]

    c = data.groupby(['route_station'])['10~11_ride'].agg({'sum', 'mean'}).reset_index()
    c.columns = ['route_station', '10~11_ride_sum','10~11_ride_mean']
    c = c[['10~11_ride_sum','10~11_ride_mean']]

    d = data.groupby(['route_station'])['10~11_takeoff'].agg({'sum', 'mean'}).reset_index()
    d.columns = ['route_station', '10~11_takeoff_sum','10~11_takeoff_mean']
    d = d[['10~11_takeoff_sum','10~11_takeoff_mean']]

    e = data.groupby(['route_station'])['11~12_ride'].agg({'sum', 'mean'}).reset_index()
    e.columns = ['route_station', '11~12_ride_sum','11~12_ride_mean']
    e = e[['11~12_ride_sum','11~12_ride_mean']]

    f = data.groupby(['route_station'])['11~12_takeoff'].agg({'sum', 'mean'}).reset_index()
    f.columns = ['route_station', '11~12_takeoff_sum','11~12_takeoff_mean']
    f = f[['11~12_takeoff_sum','11~12_takeoff_mean']]

    g = data.groupby(['route_station'])['1820_r_mean'].agg({'sum', 'mean'}).reset_index()
    g.columns = ['route_station', '1820_r_mean_sum','1820_r_mean_mean']
    g = g[['1820_r_mean_sum','1820_r_mean_mean']]

    h = data.groupby(['route_station'])['1820_r_sum'].agg({'sum', 'mean'}).reset_index()
    h.columns = ['route_station', '1820_r_sum_sum','1820_r_sum_mean']
    h = h[['1820_r_sum_sum','1820_r_sum_mean']]

    i = data.groupby(['route_station'])['1820_rs_mean'].agg({'sum', 'mean'}).reset_index()
    i.columns = ['route_station', '1820_rs_mean_sum','1820_rs_mean_mean']
    i = i[['1820_rs_mean_sum','1820_rs_mean_mean']]

    j = data.groupby(['route_station'])['1820_rs_sum'].agg({'sum', 'mean'}).reset_index()
    j.columns = ['route_station', '1820_rs_sum_sum','1820_rs_sum_mean']
    j = j[['1820_rs_sum_sum','1820_rs_sum_mean']]

    k = data.groupby(['route_station'])['1820_s_mean'].agg({'sum', 'mean'}).reset_index()
    k.columns = ['route_station', '1820_s_mean_sum','1820_s_mean_mean']
    k = k[['1820_s_mean_sum','1820_s_mean_mean']]

    l = data.groupby(['route_station'])['1820_s_sum'].agg({'sum', 'mean'}).reset_index()
    l.columns = ['route_station', '1820_s_sum_sum','1820_s_sum_mean']
    l = l[['1820_s_sum_sum','1820_s_sum_mean']]

    m = data.groupby(['route_station'])['1820_w_mean'].agg({'sum', 'mean'}).reset_index()
    m.columns = ['route_station', '1820_w_mean_sum','1820_w_mean_mean']
    m = m[['1820_w_mean_sum','1820_w_mean_mean']]

    n = data.groupby(['route_station'])['1820_w_sum'].agg({'sum', 'mean'}).reset_index()
    n.columns = ['route_station', '1820_w_sum_sum','1820_w_sum_mean']
    n = n[['1820_w_sum_sum','1820_w_sum_mean']]

    o = data.groupby(['route_station'])['68a'].agg({'sum', 'mean'}).reset_index()
    o.columns = ['route_station', '68a_sum','68a_mean']
    o = o[['68a_sum','68a_mean']]

    p = data.groupby(['route_station'])['68b'].agg({'sum', 'mean'}).reset_index()
    p.columns = ['route_station', '68b_sum','68b_mean']
    p = p[['68b_sum','68b_mean']]

    q = data.groupby(['route_station'])['6~7_ride'].agg({'sum', 'mean'}).reset_index()
    q.columns = ['route_station', '6~7_ride_sum','6~7_ride_mean']
    q = q[['6~7_ride_sum','6~7_ride_mean']]

    r = data.groupby(['route_station'])['6~7_takeoff'].agg({'sum', 'mean'}).reset_index()
    r.columns = ['route_station', '6~7_takeoff_sum','6~7_takeoff_mean']
    r = r[['6~7_takeoff_sum','6~7_takeoff_mean']]

    s = data.groupby(['route_station'])['7~8_ride'].agg({'sum', 'mean'}).reset_index()
    s.columns = ['route_station', '7~8_ride_sum','7~8_ride_mean']
    s = s[['7~8_ride_sum','7~8_ride_mean']]

    t = data.groupby(['route_station'])['7~8_takeoff'].agg({'sum', 'mean'}).reset_index()
    t.columns = ['route_station', '7~8_takeoff_sum','7~8_takeoff_mean']
    t = t[['7~8_takeoff_sum','7~8_takeoff_mean']]

    u = data.groupby(['route_station'])['810a'].agg({'sum', 'mean'}).reset_index()
    u.columns = ['route_station', '810a_sum','810a_mean']
    u = u[['810a_sum','810a_mean']]

    v = data.groupby(['route_station'])['810b'].agg({'sum', 'mean'}).reset_index()
    v.columns = ['route_station', '810b_sum','810b_mean']
    v = v[['810b_sum','810b_mean']]

    w = data.groupby(['route_station'])['8~9_ride'].agg({'sum', 'mean'}).reset_index()
    w.columns = ['route_station', '8~9_ride_sum','8~9_ride_mean']
    w = w[['8~9_ride_sum','8~9_ride_mean']]

    x = data.groupby(['route_station'])['8~9_takeoff'].agg({'sum', 'mean'}).reset_index()
    x.columns = ['route_station', '8~9_takeoff_sum','8~9_takeoff_mean']
    x = x[['8~9_takeoff_sum','8~9_takeoff_mean']]

    y = data.groupby(['route_station'])['9~10_ride'].agg({'sum', 'mean'}).reset_index()
    y.columns = ['route_station', '9~10_ride_sum','9~10_ride_mean']
    y = y[['9~10_ride_sum','9~10_ride_mean']]

    z = data.groupby(['route_station'])['9~10_takeoff'].agg({'sum', 'mean'}).reset_index()
    z.columns = ['route_station', '9~10_takeoff_sum','9~10_takeoff_mean']
    z = z[['9~10_takeoff_sum','9~10_takeoff_mean']]
    
    df = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z],axis=1)
    df = pd.merge(data, df, how='left', on='route_station')
    
    return df

In [26]:
data = morning()
data.shape

(643593, 98)

## 배차간격
배차간격 변수 --> 많이 소요
따로 파일 저장

In [27]:
train['bus_route_id'] = train['bus_route_id'].astype(np.int64)
test['bus_route_id'] = test['bus_route_id'].astype(np.int64)

In [28]:
bts

,user_card_id,bus_route_id,vhc_id,geton_date,geton_time,geton_station_code,geton_station_name,getoff_date,getoff_time,getoff_station_code,getoff_station_name,user_category,user_count,on_time
0,1.010010e+15,23000000,149793674,2019-09-10,06:34:45,360,노형오거리,2019-09-10,07:10:31,592.0,화북초등학교,1,1,6~7_ride
1,1.010010e+15,23000000,149793674,2019-09-10,06:34:58,360,노형오거리,2019-09-10,06:56:27,3273.0,고산동산(광양방면),1,1,6~7_ride
2,1.019160e+15,21420000,149793535,2019-09-10,07:19:07,2495,동광환승정류장4(제주방면),2019-09-10,07:40:29,431.0,정존마을,4,1,7~8_ride
3,1.019150e+15,21420000,149793512,2019-09-09,09:14:47,3282,대정환승정류장(대정읍사무소),2019-09-09,10:02:46,431.0,정존마을,1,1,9~10_ride
4,1.010010e+15,21420000,149793512,2019-09-09,09:28:53,2820,삼정지에듀,2019-09-09,10:21:37,2972.0,제주국제공항(종점),4,1,9~10_ride
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2409409,6.573162e+15,30420000,149797565,2019-10-16,07:08:31,1937,L마트,NaN,NaN,NaN,NaN,1,1,7~8_ride
2409410,9.441160e+15,30420000,149797565,2019-10-16,07:16:31,1908,한라산교회,NaN,NaN,NaN,NaN,1,1,7~8_ride
2409411,9.446038e+15,25070000,149797565,2019-10-16,08:29:05,1882,고도농원,NaN,NaN,NaN,NaN,1,1,8~9_ride
2409412,4.309690e+15,25070000,149797565,2019-10-16,08:40:32,1938,주공아파트5단지,NaN,NaN,NaN,NaN,1,1,8~9_ride


In [30]:
# 탑승 시간 탑승 일자 버스정류소 ID 노선 ID로 묶은 것의 user_count의 sum을 구한다.
bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

f = bts.groupby(['geton_date','geton_time2','geton_station_code','bus_route_id'])['user_count'].\
agg([('탑승객_수','sum')]).reset_index().\
sort_values(by=['geton_date','geton_station_code','bus_route_id','geton_time2'], ascending=True).reset_index()

f['index'] = list(range(0,len(f)))

In [32]:
f["탑승객_수"].value_counts()

1     2384037
2       22612
3        1812
4         547
5         128
6          75
10         44
7          34
8          14
9           8
Name: 탑승객_수, dtype: int64

In [33]:
#배차간격 계산
time = []

for i in range(0,len(f)-1):

    if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) &\
        (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) &\
        (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):

        time.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)

    else:
        time.append(0)

time.insert(0, '0')

In [34]:
#배차 간격을 초 단위로 모두 변환한 후 반환
def get_sec(time_str):

    h, m, s = time_str.split(':')

    return int(h) * 3600 + int(m) * 60 + int(s)

In [36]:
def bus_interval() :

    f['time'] = time
    f['time2'] = f['time'].astype(str).str[7:]


    interval = f.copy()
    interval['time2'] = interval['time2'].astype(str).replace('','00:00:00')
    interval['bus_route_id'] = interval['bus_route_id'].astype(object)

    time4 = []

    for i in interval['time2'] :

        time4.append(get_sec(i))

    interval['time4'] = time4
    interval['time4'] = (interval['time4'] / 60).astype(int)

    interval = interval[interval['time4'] > 3] # 간격이 3분보다 작은 것 제외 
    interval = interval[interval['time4'] < 180] # 간격이 3시간보다 큰 것 제외

    interval = interval.groupby('bus_route_id')['time4'].agg([('bus_interval', 'mean')]).reset_index()
    interval['bus_interval'] = interval['bus_interval'].astype(int)

    # 나중에 시간을 절약하기 위해 csv 파일로 저장
    interval.to_csv('bus_interval_final.csv', index = False)

    print('success.. !')

In [37]:
bus_interval()

success.. !
